In [1]:
import pandas as pd
import numpy as np
import yfinance as yf
import datetime as dt
import pickle
import requests
from bs4 import BeautifulSoup

#### Functions

In [12]:
def overnight(prices, tickers):
    """Returns a DataFrame of the overnight returns. Takes a df of the downloaded prices and a df of the tickers.
    """
    overnight = pd.DataFrame(index = ['Overnight'])
    for i in range(len(tickers)):
        ov_close = prices[tickers.index[i]][['Close']]
        ov_close = ov_close.shift(1)
        ov_open = prices[tickers.index[i]][['Open']]
        df = pd.concat([ov_close, ov_open], axis = 1)
        df['pct_change'] = (df['Open'] - df['Close'])/df['Close']
        df['cum_prod_o'] = df[['pct_change']].apply(lambda x: (x + 1).cumprod())
        overnight[tickers.index[i]] = df['cum_prod_o'].iloc[-1]
        overnight = overnight.copy()
    return overnight

In [13]:
def intraday(prices, tickers):
    """Returns a DataFrame of the intraday returns. Takes a df of the downloaded prices and a df of the tickers.
    """
    intraday = pd.DataFrame(index = ['Intraday'])
    for i in range(len(tickers)):
        in_open = prices[tickers.index[i]][['Open']]
        in_close = prices[tickers.index[i]][['Close']]
        df = pd.concat([in_open, in_close], axis = 1)
        df['pct_change'] = (df['Close'] - df['Open'])/df['Open']
        df['cum_prod_i'] = df[['pct_change']].apply(lambda x: (x + 1).cumprod())
        intraday[tickers.index[i]] = df['cum_prod_i'].iloc[-1]
        intraday = intraday.copy()
    return intraday

In [14]:
def total(prices, tickers):
    total = pd.DataFrame(index = ['Total'])
    for i in range(len(tickers)):
        df = prices[tickers.index[i]]
        open1 = df['Open'].loc[df[['Open']].first_valid_index()]
        returns = (df['Close'][-1] - open1)/open1
        total[tickers.index[i]] = returns
        total = total.copy()
    return total

In [67]:
def together(prices, tickers):
    """Combines the DataFrames from the Overnight, Intraday and Total functions. Takes a prices df and a tickers df.
    """
    def t_f(row):
        if row[-2] == 'False' and row[-1] == 'False':
            return False
        else:
            return True
    overnight_df = overnight(prices, tickers)
    intraday_df = intraday(prices, tickers)
    total_df = total(prices, tickers)
    together_df = overnight_df.append([intraday_df, total_df])
    together_df = together_df.transpose()
    together_df = tickers[['Security']].join(together_df)
    together_df['O > I'] = np.where(together_df['Overnight'] > together_df['Intraday'], 'True', 'False')
    together_df['O > T'] = np.where(together_df['Overnight'] > together_df['Total'], 'True', 'False')
    together_df['O > I and T'] = together_df.apply(lambda x: t_f(x), axis = 1)
    return together_df

#### S&P New

In [2]:
start = dt.datetime(2010, 1, 1)
end = dt.datetime.now()

In [3]:
wikiurl='https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'
table_class="wikitable sortable jquery-tablesorter"
response=requests.get(wikiurl)
print(response.status_code)

200


In [4]:
soup = BeautifulSoup(response.text, 'html.parser')
sp_table=soup.find('table',{'class':"wikitable"})

In [7]:
df=pd.read_html(str(sp_table))
df=pd.DataFrame(df[0])
df['Symbol'] = df['Symbol'].str.replace('.','-',regex = True)
df.index = df['Symbol']
df = df.drop('Symbol', axis = 1)
df

,Security,SEC filings,GICS Sector,GICS Sub-Industry,Headquarters Location,Date first added,CIK,Founded
Symbol,,,,,,,,
MMM,3M,reports,Industrials,Industrial Conglomerates,"Saint Paul, Minnesota",1976-08-09,66740,1902
AOS,A. O. Smith,reports,Industrials,Building Products,"Milwaukee, Wisconsin",2017-07-26,91142,1916
ABT,Abbott,reports,Health Care,Health Care Equipment,"North Chicago, Illinois",1964-03-31,1800,1888
ABBV,AbbVie,reports,Health Care,Pharmaceuticals,"North Chicago, Illinois",2012-12-31,1551152,2013 (1888)
ABMD,Abiomed,reports,Health Care,Health Care Equipment,"Danvers, Massachusetts",2018-05-31,815094,1981
...,...,...,...,...,...,...,...,...
YUM,Yum! Brands,reports,Consumer Discretionary,Restaurants,"Louisville, Kentucky",1997-10-06,1041061,1997
ZBRA,Zebra,reports,Information Technology,Electronic Equipment & Instruments,"Lincolnshire, Illinois",2019-12-23,877212,1969
ZBH,Zimmer Biomet,reports,Health Care,Health Care Equipment,"Warsaw, Indiana",2001-08-07,1136869,1927


In [8]:
%%time
dic = {}
for ticker in df.index:
    dic["{}".format(ticker)] = yf.download(ticker, start = start, end = end)[['Open', 'Close']]
dic

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

{'MMM':                   Open       Close
 Date                              
 2010-01-04   83.089996   83.019997
 2010-01-05   82.800003   82.500000
 2010-01-06   83.879997   83.669998
 2010-01-07   83.320000   83.730003
 2010-01-08   83.690002   84.320000
 ...                ...         ...
 2022-05-18  151.589996  148.929993
 2022-05-19  146.199997  146.960007
 2022-05-20  147.539993  143.830002
 2022-05-23  144.580002  144.380005
 2022-05-24  144.350006  145.750000
 
 [3120 rows x 2 columns],
 'AOS':                  Open      Close
 Date                            
 2010-01-04   7.295000   7.435000
 2010-01-05   7.431667   7.340000
 2010-01-06   7.335000   7.343333
 2010-01-07   7.356667   7.365000
 2010-01-08   7.331667   7.473333
 ...               ...        ...
 2022-05-18  59.349998  57.770000
 2022-05-19  57.230000  58.450001
 2022-05-20  58.900002  57.770000
 2022-05-23  58.500000  58.369999
 2022-05-24  58.320000  58.349998
 
 [3120 rows x 2 columns],
 'ABT':             

In [9]:
sp_new = pd.concat(dic, axis = 1)
sp_new

MMM                    AOS                    ABT  \
                  Open       Close       Open      Close        Open   
Date                                                                   
2010-01-04   83.089996   83.019997   7.295000   7.435000   26.000362   
2010-01-05   82.800003   82.500000   7.431667   7.340000   26.134706   
2010-01-06   83.879997   83.669998   7.335000   7.343333   25.880411   
2010-01-07   83.320000   83.730003   7.356667   7.365000   26.057938   
2010-01-08   83.690002   84.320000   7.331667   7.473333   26.273848   
...                ...         ...        ...        ...         ...   
2022-05-18  151.589996  148.929993  59.349998  57.770000  113.419998   
2022-05-19  146.199997  146.960007  57.230000  58.450001  112.029999   
2022-05-20  147.539993  143.830002  58.900002  57.770000  113.400002   
2022-05-23  144.580002  144.380005  58.500000  58.369999  113.849998   
2022-05-24  144.350006  145.750000  58.320000  58.349998  115.160004   

                              ABBV                    ABMD              ...  \
                 Close        Open       Close        Open       Close  ...   
Date                                                                    ...   
2010-01-04   26.129908         NaN         NaN    8.860000    8.740000  ...   
2010-01-05   25.918797         NaN         NaN    8.740000    8.530000  ...   
2010-01-06   26.062737         NaN         NaN    8.520000    8.400000  ...   
2010-01-07   26.278646         NaN         NaN    8.630000    8.400000  ...   
2010-01-08   26.412991         NaN         NaN    8.380000    8.230000  ...   
...                ...         ...         ...         ...         ...  ...   
2022-05-18  112.739998  154.789993  152.429993  253.220001  240.460007  ...   
2022-05-19  112.440002  151.199997  151.720001  239.300003  238.649994  ...   
2022-05-20  113.239998  152.330002  151.009995  241.059998  235.750000  ...   
2022-05-23  115.029999  151.649994  148.029999  238.300003  249.220001  ...   
2022-05-24  113.769997  147.029999  149.110001  244.690002  249.190002  ...   

                   YUM                    ZBRA                     ZBH  \
                  Open       Close        Open       Close        Open   
Date                                                                     
2010-01-04   25.248022   25.226456   28.469999   28.670000   57.582523   
2010-01-05   25.248022   25.140186   28.549999   28.620001   59.533981   
2010-01-06   25.125809   24.960461   27.410000   28.400000   60.252426   
2010-01-07   24.852625   24.953272   28.230000   27.690001   61.019417   
2010-01-08   24.946081   24.960461   27.580000   27.600000   60.009708   
...                ...         ...         ...         ...         ...   
2022-05-18  111.400002  109.459999  338.829987  326.420013  118.000000   
2022-05-19  109.080002  109.970001  324.170013  324.100006  115.089996   
2022-05-20  110.989998  111.919998  330.019989  325.369995  118.320000   
2022-05-23  113.000000  113.820000  330.160004  334.730011  119.239998   
2022-05-24  113.269997  114.070000  330.299988  322.880005  120.199997   

                             ZION                    ZTS              
                 Close       Open      Close        Open       Close  
Date                                                                  
2010-01-04   58.271847  12.940000  13.330000         NaN         NaN  
2010-01-05   60.116505  13.280000  13.800000         NaN         NaN  
2010-01-06   60.097088  13.670000  15.000000         NaN         NaN  
2010-01-07   61.475727  14.970000  16.680000         NaN         NaN  
2010-01-08   60.184467  16.190001  16.410000         NaN         NaN  
...                ...        ...        ...         ...         ...  
2022-05-18  116.449997  54.040001  53.000000  164.800003  158.860001  
2022-05-19  116.910004  52.270000  52.060001  158.139999  158.630005  
2022-05-20  117.610001  52.599998  51.990002  161.610001  162.559998  
2022-05-23  12

In [10]:
with open('sp_new.txt', "wb") as file:
    pickle.dump(sp_new, file)

In [11]:
with open('sp_new.txt', "rb") as file:
    sp_new = pickle.load(file)
sp_new

MMM                    AOS                    ABT  \
                  Open       Close       Open      Close        Open   
Date                                                                   
2010-01-04   83.089996   83.019997   7.295000   7.435000   26.000362   
2010-01-05   82.800003   82.500000   7.431667   7.340000   26.134706   
2010-01-06   83.879997   83.669998   7.335000   7.343333   25.880411   
2010-01-07   83.320000   83.730003   7.356667   7.365000   26.057938   
2010-01-08   83.690002   84.320000   7.331667   7.473333   26.273848   
...                ...         ...        ...        ...         ...   
2022-05-18  151.589996  148.929993  59.349998  57.770000  113.419998   
2022-05-19  146.199997  146.960007  57.230000  58.450001  112.029999   
2022-05-20  147.539993  143.830002  58.900002  57.770000  113.400002   
2022-05-23  144.580002  144.380005  58.500000  58.369999  113.849998   
2022-05-24  144.350006  145.750000  58.320000  58.349998  115.160004   

                              ABBV                    ABMD              ...  \
                 Close        Open       Close        Open       Close  ...   
Date                                                                    ...   
2010-01-04   26.129908         NaN         NaN    8.860000    8.740000  ...   
2010-01-05   25.918797         NaN         NaN    8.740000    8.530000  ...   
2010-01-06   26.062737         NaN         NaN    8.520000    8.400000  ...   
2010-01-07   26.278646         NaN         NaN    8.630000    8.400000  ...   
2010-01-08   26.412991         NaN         NaN    8.380000    8.230000  ...   
...                ...         ...         ...         ...         ...  ...   
2022-05-18  112.739998  154.789993  152.429993  253.220001  240.460007  ...   
2022-05-19  112.440002  151.199997  151.720001  239.300003  238.649994  ...   
2022-05-20  113.239998  152.330002  151.009995  241.059998  235.750000  ...   
2022-05-23  115.029999  151.649994  148.029999  238.300003  249.220001  ...   
2022-05-24  113.769997  147.029999  149.110001  244.690002  249.190002  ...   

                   YUM                    ZBRA                     ZBH  \
                  Open       Close        Open       Close        Open   
Date                                                                     
2010-01-04   25.248022   25.226456   28.469999   28.670000   57.582523   
2010-01-05   25.248022   25.140186   28.549999   28.620001   59.533981   
2010-01-06   25.125809   24.960461   27.410000   28.400000   60.252426   
2010-01-07   24.852625   24.953272   28.230000   27.690001   61.019417   
2010-01-08   24.946081   24.960461   27.580000   27.600000   60.009708   
...                ...         ...         ...         ...         ...   
2022-05-18  111.400002  109.459999  338.829987  326.420013  118.000000   
2022-05-19  109.080002  109.970001  324.170013  324.100006  115.089996   
2022-05-20  110.989998  111.919998  330.019989  325.369995  118.320000   
2022-05-23  113.000000  113.820000  330.160004  334.730011  119.239998   
2022-05-24  113.269997  114.070000  330.299988  322.880005  120.199997   

                             ZION                    ZTS              
                 Close       Open      Close        Open       Close  
Date                                                                  
2010-01-04   58.271847  12.940000  13.330000         NaN         NaN  
2010-01-05   60.116505  13.280000  13.800000         NaN         NaN  
2010-01-06   60.097088  13.670000  15.000000         NaN         NaN  
2010-01-07   61.475727  14.970000  16.680000         NaN         NaN  
2010-01-08   60.184467  16.190001  16.410000         NaN         NaN  
...                ...        ...        ...         ...         ...  
2022-05-18  116.449997  54.040001  53.000000  164.800003  158.860001  
2022-05-19  116.910004  52.270000  52.060001  158.139999  158.630005  
2022-05-20  117.610001  52.599998  51.990002  161.610001  162.559998  
2022-05-23  12

In [17]:
sp_overnight = overnight(sp_new, df)
sp_overnight

,MMM,AOS,ABT,ABBV,ABMD,ACN,ATVI,ADM,ADBE,ADP,...,WMB,WTW,WYNN,XEL,XYL,YUM,ZBRA,ZBH,ZION,ZTS
Overnight,1.353127,5.238618,1.842547,1.442071,3.364738,0.543741,9.608467,2.169149,2.69774,0.936958,...,1.945243,0.404311,12.616967,0.807711,2.478884,1.474047,3.500933,0.932736,3.160868,NaN


In [19]:
sp_intraday = intraday(sp_new, df)
sp_intraday

,MMM,AOS,ABT,ABBV,ABMD,ACN,ATVI,ADM,ADBE,ADP,...,WMB,WTW,WYNN,XEL,XYL,YUM,ZBRA,ZBH,ZION,ZTS
Intraday,1.296347,1.526859,2.362906,2.961051,8.372108,12.280429,0.713881,1.276241,4.030731,5.924146,...,1.057146,7.093093,0.075178,4.398752,1.376435,3.032909,3.203255,2.224642,1.300415,NaN


In [21]:
sp_total = total(sp_new, df)
sp_total

,MMM,AOS,ABT,ABBV,ABMD,ACN,ATVI,ADM,ADBE,ADP,...,WMB,WTW,WYNN,XEL,XYL,YUM,ZBRA,ZBH,ZION,ZTS
Total,0.754122,6.998629,3.375708,3.270046,27.125284,5.727119,5.859306,1.787802,9.870668,4.564986,...,1.071761,1.870385,-0.049804,2.566885,2.487234,3.517978,10.341061,1.073372,3.137558,NaN


In [68]:
%%time
sp_together = together(sp_new, df)
sp_together

Wall time: 7.88 s


,Security,Overnight,Intraday,Total,O > I,O > T,O > I and T
Symbol,,,,,,,
MMM,3M,1.353127,1.296347,0.754122,True,True,True
AOS,A. O. Smith,5.238618,1.526859,6.998629,True,False,True
ABT,Abbott,1.842547,2.362906,3.375708,False,False,False
ABBV,AbbVie,1.442071,2.961051,3.270046,False,False,False
ABMD,Abiomed,3.364738,8.372108,27.125284,False,False,False
...,...,...,...,...,...,...,...
YUM,Yum! Brands,1.474047,3.032909,3.517978,False,False,False
ZBRA,Zebra,3.500933,3.203255,10.341061,True,False,True
ZBH,Zimmer Biomet,0.932736,2.224642,1.073372,False,False,False


In [52]:
sp_together['O > I'].value_counts()

True     270
False    234
Name: O > I, dtype: int64

In [53]:
sp_together['O > T'].value_counts()

False    296
True     208
Name: O > T, dtype: int64

In [69]:
sp_together['O > I and T'].value_counts()

True     297
False    207
Name: O > I and T, dtype: int64

In [70]:
normal = sp_together.loc[sp_together['O > I and T'] == False]
normal

,Security,Overnight,Intraday,Total,O > I,O > T,O > I and T
Symbol,,,,,,,
ABT,Abbott,1.842547,2.362906,3.375708,False,False,False
ABBV,AbbVie,1.442071,2.961051,3.270046,False,False,False
ABMD,Abiomed,3.364738,8.372108,27.125284,False,False,False
ACN,Accenture,0.543741,12.280429,5.727119,False,False,False
ADBE,Adobe,2.697740,4.030731,9.870668,False,False,False
...,...,...,...,...,...,...,...
WTW,Willis Towers Watson,0.404311,7.093093,1.870385,False,False,False
XEL,Xcel Energy,0.807711,4.398752,2.566885,False,False,False
YUM,Yum! Brands,1.474047,3.032909,3.517978,False,False,False
